# Topic 5: Visualizations

---

## Import Required Packages

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from datetime import datetime
import xarray as xr
import rioxarray
import rasterio as rio        # https://rasterio.readthedocs.io/en/latest/
from rasterio.plot import show
import hvplot.xarray
import holoviews as hv
import pickle

**List geoTIFF files in `data` directory**

In [ ]:
file_list = [f'./data/{x}' for x in os.listdir('./data') if '.tif' in x]
file_list

**Load Red, Blue, and Green COG files (stored locally)**

In [ ]:
red = rioxarray.open_rasterio('./data/HLS.S30.T13TGF.2020191T172901.v1.5.B04.tif')
red

In [ ]:
blue = rioxarray.open_rasterio('./data/HLS.S30.T13TGF.2020191T172901.v1.5.B02.tif')
green = rioxarray.open_rasterio('./data/HLS.S30.T13TGF.2020191T172901.v1.5.B03.tif')

## Matplotlib

**rasterio and xarray plotting leverages the matplotlib library**

In [ ]:
red.squeeze().plot.imshow()

In [ ]:
# Initialize subplots
fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, nrows=1, figsize=(14,7), sharey=True)

# Plot Red, Green and Blue (rgb)
show((red.squeeze()), cmap='Reds', ax=ax1)
show((green.squeeze()), cmap='Greens', ax=ax2)
show((blue.squeeze()), cmap='Blues', ax=ax3)

# Add titles
ax1.set_title("Red")
ax2.set_title("Green")
ax3.set_title("Blue")

In [ ]:
# Read the grid values into numpy arrays
r = red.squeeze().values
g = green.squeeze().values
b = blue.squeeze().values

# Function to normalize the grid values
def normalize(array):
    """Normalizes numpy arrays into scale 0.0 - 1.0"""
    array_min, array_max = array.min(), array.max()
    return ((array - array_min)/(array_max - array_min))

# Normalize the bands
redn = normalize(r)
greenn = normalize(g)
bluen = normalize(b)

print("Normalized bands")
print(redn.min(), '-', redn.max(), 'mean:', redn.mean())
print(greenn.min(), '-', greenn.max(), 'mean:', greenn.mean())
print(bluen.min(), '-', bluen.max(), 'mean:', bluen.mean())

In [ ]:
import numpy as np
# Create RGB natural color composite
rgb = np.dstack((redn, greenn, bluen))

# Let's see how our color composite looks like
plt.figure(figsize = (20,10))
plt.imshow(rgb)

## [HoloViz](https://holoviz.org/)

![HoloViz Maintained Packages](img/pn_hp_hv_gv_ds_pa_cs.png)

In [ ]:
red

In [ ]:
red[0].hvplot.image()

In [ ]:
hls_proj = red.rio.crs.to_string()

In [ ]:
red[0].hvplot.image(x='x', y='y', crs = hls_proj, cmap='Reds', width=800, height=600, colorbar=True, tiles = 'ESRI')

In [ ]:
# Load polygon from disc
with open('./data/fsUTM', "rb") as poly_file:
    fsUTM = pickle.load(poly_file)

In [ ]:
red_plot = red[0].rio.clip([fsUTM]).hvplot.image(x='x', y='y', crs = hls_proj, cmap='Reds', width=800, height=600, colorbar=True,  tiles = 'ESRI')
red_plot

In [ ]:
import pandas as pd
chunks=dict(band=1, x=1024, y=1024)
red_ts = rioxarray.open_rasterio('data/timeseries/red_stack.vrt')
red_ts

In [ ]:
red_ts = red_ts.rename({'band':'time'})
file_list = [f for f in os.listdir('data/timeseries/') if '.tif' in f]
time_j_list = [x.split('.')[-5].split('T')[0] for x in file_list]
red_ts['time'] = [datetime.strptime(t, '%Y%j') for t in time_j_list]

In [ ]:
red_ts

In [ ]:
red_ts.hvplot.image(x='x', y='y', crs = hls_proj, cmap='Reds', width=800, height=600, tiles='ESRI')